In [21]:
from find_source import full_html_and_txt, incl_excl_data, make_catalog, summary, get_prob_image_rms
import glob
import json

In [22]:
#%%timeit
full_html_and_txt('../data/250611_03:56:34')

In [ ]:
folders = []
for folder in glob.glob('/reduction/karto/SMA/polaris_test/*'):
    print(folder)
    folders.append(folder)

/reduction/karto/SMA/polaris_test/250101_04:01:33
/reduction/karto/SMA/polaris_test/run_polaris.m
/reduction/karto/SMA/polaris_test/250101_14:31:46
/reduction/karto/SMA/polaris_test/250102_03:39:23
/reduction/karto/SMA/polaris_test/250103_05:25:18
/reduction/karto/SMA/polaris_test/250104_03:18:27
/reduction/karto/SMA/polaris_test/250105_04:05:43
/reduction/karto/SMA/polaris_test/250106_03:59:50
/reduction/karto/SMA/polaris_test/250106_13:01:37
/reduction/karto/SMA/polaris_test/250106_15:00:08
/reduction/karto/SMA/polaris_test/250106_17:38:06
/reduction/karto/SMA/polaris_test/250107_03:21:27
/reduction/karto/SMA/polaris_test/250107_11:09:13
/reduction/karto/SMA/polaris_test/250108_04:00:57
/reduction/karto/SMA/polaris_test/250109_03:31:04
/reduction/karto/SMA/polaris_test/250109_07:40:11
/reduction/karto/SMA/polaris_test/250109_15:20:50
/reduction/karto/SMA/polaris_test/250111_03:59:34
/reduction/karto/SMA/polaris_test/250112_04:00:37
/reduction/karto/SMA/polaris_test/250114_03:31:47
/r

In [ ]:
full_html_and_txt(folders[0])
print(folders[0])
folders = folders[1:]

/home/cxu/repo/protostar/src/find_source.py:766: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(6.7,5.1))


/reduction/karto/SMA/polaris_test/250106_03:59:50


In [ ]:
'''
Using the exclusion region's rms taken directly from the image,
finds the probability of detecting the inclusion region's maximum flux if there were no source in the inclusion region,
the probability of detecting the exclusion region's maximum flux if there were no source in the exclusion region, and other statistics.

If the external probability is less than 0.001, updates the inclusion region to include a circle around the external peak.

The other statisitcs are the inclusion region's maximum flux in Jy and its coordinates in pixels,
the exclusion region's maximum flux in Jy and its coordinates in pixels, the exclusion region's rms in Jy,
the number of measurements in the inclusion region, the number of measurements in the exclusion region,
the coordinates in pixels of the image's center, and the radii in pixels of the inclusion zones,
the inclusion region's signal to noise ratio, and the external region's signal to noise ratio.

Parameters
----------
fits_file : str
    The path of the FITS file that contains the image.
center : list (optional)
    A list of center coordinates in units of pixels.
    If no center coordinates are given, eventually defaults to ((length of x-axis)/2, (length of y-axis)/2), rounded up.
radius_buffer : float (optional)
    The amount of buffer, in arcsec, to add to the beam FWHM to get the initial search radius.
    If no value is given, defaults to 5 arcsec.
ext_threshold : float (optional)
    The probability that an external peak must be below for it to be considered an external source.
    If no value is given, defaults to 0.001.
rms : float (optional)
    An rms value in Jy.
    If no value is given, eventually defaults to the rms calculated by incl_excl_data.
recursion : bool (optional)
    Whether to use recursion to find significant external peaks, if any.
    If no value is given, defaults to True.

Returns
-------
list
    A list with:
        dict (possibly multiple)
            A dictionary with:
                float
                    The probability of detecting the inclusion region's maximum flux if there were no source in the inclusion region.
                float
                    The probability of detecting the exclusion region's maximum flux if there were no source in the exclusion region.
                float
                    The inclusion region's maximum flux in Jy.
                tuple (int, int)
                    The coordinates in pixels of the inclusion region's maximum flux.
                float
                    The exclusion region's maximum flux in Jy.
                tuple (int, int)
                    The coordinates in pixels of the exclusion region's maximum flux.
                float
                    The exclusion region's rms in Jy.
                float
                    The number of measurements in the inclusion region.
                float
                    The number of measurements in the exclusion region.
                tuple (int, int)
                    The coordinates in pixels of the image's center.
                list
                    A list with:
                        float(s)
                            The radii in pixels of inclusion zones.
                float
                    The inclusion region's signal to noise ratio.
                float
                    The exclusion region's signal to noise ratio.
'''

In [ ]:
'''
Finds statistics of an inclusion region and its complement, the exclusion region.

The inclusion region can be the union of circles or the complement of such a region.

The statistics are the inclusion region's maximum flux in Jy and its coordinates in pixels,
the exclusion region's maximum flux in Jy and its coordinates in pixels, the exclusion region's rms in Jy,
the number of measurements in the inclusion region, the number of measurements in the exclusion region,
the coordinates in pixels of the image's center, and the radii in pixels of the inclusion zones.

Parameters
----------
fits_file : str
    The path of the FITS file that contains the image.
center : list (optional)
    A list of center coordinates in units of pixels.
    If no center coordinates are given, eventually defaults to ((length of x-axis)/2, (length of y-axis)/2), rounded up.
radius_buffer : float (optional)
    The amount of buffer, in arcsec, to add to the beam FWHM to get the initial search radius.
    If no value is given, defaults to 5 arcsec.

Returns
-------
dict
    A dictionary with:
        float
            The inclusion region's maximum flux in Jy.
        tuple (int, int)
            The coordinates in pixels of the inclusion region's maximum flux.
        float
            The exclusion region's maximum flux in Jy.
        tuple (int, int)
            The coordinates in pixels of the exclusion region's maximum flux.
        float
            The exclusion region's rms in Jy.
        float
            The number of measurements in the inclusion region.
        float
            The number of measurements in the exclusion region.
        tuple (int, int)
            The coordinates in pixels of the image's center.
        list
            A list with:
                float(s)
                    The radii in pixels of inclusion zones.
'''